In [138]:
from enum import Enum

import pandas as pd

from plot_utils import *

## Индекс потребительских цен на товары и услуги

In [139]:
class IpcType(Enum):
    """Возможные страницы Excel-файла с индексами потребительских цен (ИПЦ или CPI)."""
    GOODS_AND_SERVICES = 1  # Потребительские цены на товары и услуги
    FOOD = 2  # Потребительские цены на продовольственные товары
    NONFOOD = 3  # Потребительские цены на непродовольственные товары
    SERVICES = 4  # Потребительские цены на услуги


def load_ipc_data(ipc_type: IpcType = IpcType.GOODS_AND_SERVICES) -> pd.DataFrame:
    """
    Загружает и обрабатывает указанный лист Excel-файла с данными ИПЦ.
    Табличный формат удобен для восприятия и обработки.
    """
    df = pd.read_excel('./stats/ipc_mes_08-2025.xlsx', sheet_name=ipc_type.value, skiprows=3)
    df = df.iloc[1:13]
    df = df.drop(df.columns[1:10], axis=1)
    df.columns.values[0] = 'месяц'
    return df


def load_melted_ipc_data(ipc_type: IpcType = IpcType.GOODS_AND_SERVICES) -> pd.DataFrame:
    """
    Возвращает преобразованный DataFrame с колонками 'date' и 'cpi'.
    Сплющенный формат, вместо табличного, удобнее для построения графиков.
    """
    df = load_ipc_data(ipc_type)

    # --- "Расплавляем" таблицу ---
    df_melted = df.melt(id_vars='месяц', var_name='год', value_name='cpi')
    df_melted = df_melted.dropna(subset=['cpi'])

    month_map = {
        'январь': 1, 'февраль': 2, 'март': 3, 'апрель': 4,
        'май': 5, 'июнь': 6, 'июль': 7, 'август': 8,
        'сентябрь': 9, 'октябрь': 10, 'ноябрь': 11, 'декабрь': 12
    }
    df_melted['month_num'] = df_melted['месяц'].map(month_map)
    df_melted['год'] = df_melted['год'].astype(int)

    df_melted['date'] = pd.to_datetime(
        df_melted['год'].astype(str) + '-' + df_melted['month_num'].astype(str) + '-01'
    )

    # --- Выбираем только нужные столбцы ---
    df_melted = df_melted[['date', 'cpi']].reset_index(drop=True)
    return df_melted

In [140]:
df = load_ipc_data(IpcType.GOODS_AND_SERVICES)
df

,месяц,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
1,январь,102.33,102.76,103.09,102.40,101.75,102.62,102.43,101.68,102.31,...,100.96,100.62,100.31,101.01,100.40,100.67,100.99,100.84,100.86,101.23
2,февраль,101.04,102.28,101.16,101.63,100.99,101.23,101.66,101.11,101.20,...,100.63,100.22,100.21,100.44,100.33,100.78,101.17,100.46,100.68,100.81
3,март,100.64,101.86,101.08,101.05,100.75,101.34,100.82,100.59,101.20,...,100.46,100.13,100.29,100.32,100.55,100.66,107.61,100.37,100.39,100.65
4,апрель,100.89,101.79,101.16,101.02,100.99,101.12,100.35,100.57,101.42,...,100.44,100.33,100.38,100.29,100.83,100.58,101.56,100.38,100.50,100.4
5,май,101.75,101.78,101.69,100.80,100.74,100.80,100.48,100.63,101.35,...,100.41,100.37,100.38,100.34,100.27,100.74,100.12,100.31,100.74,100.43
6,июнь,102.55,101.62,100.53,100.80,100.78,100.64,100.28,100.95,100.97,...,100.36,100.61,100.49,100.04,100.22,100.69,99.65,100.37,100.64,100.2
7,июль,101.79,100.45,100.72,100.71,100.92,100.46,100.67,100.87,100.51,...,100.54,100.07,100.27,100.20,100.35,100.31,99.61,100.63,101.14,100.57
8,август,100.98,100.01,100.09,99.59,100.42,99.86,100.19,100.09,100.36,...,100.01,99.46,100.01,99.76,99.96,100.17,99.48,100.28,100.20,99.6
9,сентябрь,101.32,100.60,100.40,100.34,100.43,100.25,100.09,100.79,100.80,...,100.17,99.85,100.16,99.84,99.93,100.60,100.05,100.87,100.48,NaN
10,октябрь,102.11,101.09,101.07,101.00,101.14,100.55,100.28,101.64,100.91,...,100.43,100.20,100.35,100.13,100.43,101.11,100.18,100.83,100.75,NaN


In [141]:
df_melted = load_melted_ipc_data(IpcType.GOODS_AND_SERVICES)
df_melted.head()

,date,cpi
0,2000-01-01,102.33
1,2000-02-01,101.04
2,2000-03-01,100.64
3,2000-04-01,100.89
4,2000-05-01,101.75


In [142]:
plot_2d_interactive(
    x=df_melted['date'],
    y=df_melted['cpi'],
    xlabel="Дата",
    ylabel="Отношение к предыдущему месяцу (в процентах)",
    title=f"Индекс потребительских цен в России на товары и услуги (1991–2025)"
)

In [143]:
def load_normalized_ipc_data(
    ipc_type: IpcType = IpcType.GOODS_AND_SERVICES,
    base_year: str | pd.Timestamp = '2000-01-01'
) -> pd.DataFrame:
    """
    Загружает cpi, рассчитывает накопленный индекс и реальную покупательную способность рубля относительно базового года.
    """
    df = load_melted_ipc_data(ipc_type)

    base_date = pd.Timestamp(base_year)

    df['factor'] = df['cpi'] / 100
    df['cumulative_cpi'] = df['factor'].cumprod()

    # нормализация к базовому году
    base_value = df.loc[df['date'] == base_date, 'cumulative_cpi'].values[0]

    df['cpi_base'] = (df['cumulative_cpi'] / base_value) * 100

    df['real_ruble'] = 100 / df['cpi_base']

    return df[['date', 'cpi', 'cumulative_cpi', 'real_ruble']]

df_melted = load_normalized_ipc_data(IpcType.GOODS_AND_SERVICES, base_year='2000-01-01')
df_melted.head()

,date,cpi,cumulative_cpi,real_ruble
0,2000-01-01,102.33,1.0233,1.0
1,2000-02-01,101.04,1.033942,0.989707
2,2000-03-01,100.64,1.04056,0.983413
3,2000-04-01,100.89,1.049821,0.974738
4,2000-05-01,101.75,1.068192,0.957973


In [144]:
plot_2d_interactive(
    x=df_melted['date'],
    y=df_melted['real_ruble'],
    xlabel="Дата",
    ylabel="Реальная покупательная способность (в ценах 1 января 2000 г.)",
    title="Покупательная способность рубля относительно базового уровня на товары и услуги (2000 = 1.0)"
)

## Индекс потребительских цен на продовольственные товары

In [145]:
df = load_ipc_data(IpcType.FOOD)
df.head(12)

,месяц,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
1,январь,102.17,103.06,102.82,102.51,101.61,101.42,101.97,100.94,101.91,...,101.19,100.85,100.50,101.27,100.69,101.01,101.44,101.32,101.26,101.33
2,февраль,100.46,102.28,100.87,101.20,101.12,101.37,102.98,100.83,101.66,...,100.68,100.20,100.35,100.79,100.57,101.22,101.56,100.79,100.77,101.27
3,март,100.09,101.77,100.46,100.97,101.05,102.07,101.17,100.77,101.98,...,100.40,100.14,100.53,100.53,100.96,100.82,106.73,100.13,100.17,100.83
4,апрель,100.30,102.01,100.96,100.98,100.84,101.70,100.26,100.75,102.19,...,100.42,100.60,100.43,100.43,101.72,100.75,102.87,100.29,100.49,100.71
5,май,102.17,102.27,102.21,100.69,100.43,101.06,100.47,100.95,102.07,...,100.37,100.59,99.94,100.41,100.16,100.96,100.60,99.69,100.41,100.26
6,июнь,103.26,101.91,99.82,100.79,100.80,100.72,100.02,101.73,101.08,...,100.12,101.02,100.38,99.52,100.17,100.64,98.90,99.99,100.63,100.11
7,июль,101.75,99.74,100.43,100.41,101.01,100.27,100.89,101.42,100.09,...,99.96,99.03,99.67,99.70,99.93,99.50,98.47,100.49,100.36,99.36
8,август,100.29,98.97,98.92,98.59,100.11,98.95,99.45,99.40,99.82,...,99.41,98.20,99.62,99.09,99.23,99.49,98.64,99.94,99.99,99.09
9,сентябрь,100.60,99.75,99.27,99.82,100.04,99.34,99.37,101.02,100.74,...,99.86,99.29,99.92,99.56,99.59,100.99,99.62,100.86,100.34,NaN
10,октябрь,102.13,100.74,100.62,101.10,101.39,100.36,99.95,103.26,101.57,...,100.78,100.38,100.55,100.18,100.62,102.17,100.28,101.35,101.23,NaN


In [146]:
df_melted = load_melted_ipc_data(IpcType.FOOD)
df_melted.head()

,date,cpi
0,2000-01-01,102.17
1,2000-02-01,100.46
2,2000-03-01,100.09
3,2000-04-01,100.3
4,2000-05-01,102.17


In [147]:
plot_2d_interactive(
    x=df_melted['date'],
    y=df_melted['cpi'],
    xlabel="Дата",
    ylabel="Отношение к предыдущему месяцу (в процентах)",
    title=f"Индекс потребительских цен в России на продовольственные товары (1991–2025)"
)

In [148]:
df_melted = load_normalized_ipc_data(IpcType.FOOD, base_year='2000-01-01')
df_melted

,date,cpi,cumulative_cpi,real_ruble
0,2000-01-01,102.17,1.0217,1.0
1,2000-02-01,100.46,1.0264,0.995421
2,2000-03-01,100.09,1.027324,0.994526
3,2000-04-01,100.3,1.030406,0.991551
4,2000-05-01,102.17,1.052765,0.970492
...,...,...,...,...
303,2025-04-01,100.71,10.607076,0.096322
304,2025-05-01,100.26,10.634654,0.096073
305,2025-06-01,100.11,10.646352,0.095967
306,2025-07-01,99.36,10.578216,0.096585


In [149]:
plot_2d_interactive(
    x=df_melted['date'],
    y=df_melted['real_ruble'],
    xlabel="Дата",
    ylabel="Реальная покупательная способность (в ценах 1 января 2000 г.)",
    title="Покупательная способность рубля относительно базового уровня на продовольственные товары (2000 = 1.0)"
)

## Индекс потребительских цен на непродовольственные товары

In [150]:
df = load_ipc_data(IpcType.NONFOOD)
df.head(12)

,месяц,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
1,январь,102.17,101.35,101.23,101.08,100.51,100.37,100.41,100.40,100.62,...,100.67,100.47,100.30,100.64,100.23,100.53,100.67,100.21,100.47,100.42
2,февраль,101.32,101.34,100.77,100.88,100.42,100.35,100.45,100.33,100.59,...,100.78,100.20,100.13,100.26,100.04,100.58,100.78,99.92,100.26,100.26
3,март,101.38,101.25,100.68,100.81,100.44,100.39,100.42,100.38,100.74,...,100.81,100.22,100.16,100.25,100.48,100.72,111.25,100.14,100.27,100.14
4,апрель,101.49,100.87,100.77,100.63,100.58,100.51,100.28,100.40,100.88,...,100.60,100.17,100.37,100.19,100.44,100.66,100.53,100.15,100.41,99.93
5,май,101.09,100.87,101.19,100.58,100.77,100.43,100.41,100.35,100.79,...,100.42,100.15,100.88,100.21,100.25,100.74,99.91,100.34,100.46,99.87
6,июнь,100.83,100.59,101.13,100.54,100.67,100.31,100.34,100.30,100.74,...,100.46,100.11,100.41,100.17,100.34,100.68,99.59,100.42,100.29,99.97
7,июль,100.84,100.53,100.57,100.47,100.56,100.43,100.40,100.37,100.69,...,100.37,100.08,100.12,100.17,100.29,100.77,99.56,100.91,100.58,100.19
8,август,101.38,100.79,100.68,100.58,100.52,100.53,100.76,100.60,100.57,...,100.41,100.14,100.22,100.17,100.41,100.80,99.95,101.14,100.60,100.42
9,сентябрь,102.05,101.16,100.87,100.87,100.87,101.09,100.76,100.77,100.67,...,100.56,100.25,100.36,100.19,100.56,100.64,100.15,101.09,100.62,NaN
10,октябрь,101.89,101.31,100.91,100.96,100.74,100.71,100.61,100.86,100.81,...,100.54,100.30,100.45,100.31,100.68,100.78,100.04,100.55,100.68,NaN


In [151]:
df_melted = load_melted_ipc_data(IpcType.NONFOOD)
df_melted.head()

,date,cpi
0,2000-01-01,102.17
1,2000-02-01,101.32
2,2000-03-01,101.38
3,2000-04-01,101.49
4,2000-05-01,101.09


In [152]:
plot_2d_interactive(
    x=df_melted['date'],
    y=df_melted['cpi'],
    xlabel="Дата",
    ylabel="Отношение к предыдущему месяцу (в процентах)",
    title=f"Индекс потребительских цен в России на непродовольственные товары (1991–2025)"
)

In [153]:
df_melted = load_normalized_ipc_data(IpcType.NONFOOD, base_year='2000-01-01')
df_melted

,date,cpi,cumulative_cpi,real_ruble
0,2000-01-01,102.17,1.0217,1.0
1,2000-02-01,101.32,1.035186,0.986972
2,2000-03-01,101.38,1.049472,0.973537
3,2000-04-01,101.49,1.065109,0.959244
4,2000-05-01,101.09,1.076719,0.948901
...,...,...,...,...
303,2025-04-01,99.93,6.356561,0.160732
304,2025-05-01,99.87,6.348297,0.160941
305,2025-06-01,99.97,6.346393,0.160989
306,2025-07-01,100.19,6.358451,0.160684


In [154]:
plot_2d_interactive(
    x=df_melted['date'],
    y=df_melted['real_ruble'],
    xlabel="Дата",
    ylabel="Реальная покупательная способность (в ценах 1 января 2000 г.)",
    title="Покупательная способность рубля относительно базового уровня на непродовольственные товары (2000 = 1.0)"
)

## Индексы потребительских цен на услуги

In [155]:
df = load_ipc_data(IpcType.SERVICES)
df.head(12)

,месяц,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
1,январь,103.37,104.63,107.47,104.42,104.13,108.78,106.17,104.71,105.40,...,101.03,100.51,100.08,101.13,100.24,100.38,100.76,101.01,100.78,102.07
2,февраль,102.97,104.27,102.73,104.16,101.63,102.23,101.01,102.60,101.36,...,100.34,100.28,100.12,100.20,100.37,100.44,101.13,100.72,101.06,100.81
3,март,101.45,103.44,103.70,101.67,100.56,101.22,100.72,100.57,100.63,...,100.06,100.00,100.12,100.13,100.09,100.37,103.99,100.97,100.83,101.01
4,апрель,102.14,102.80,102.40,101.78,101.99,100.82,100.58,100.51,100.96,...,100.25,100.16,100.32,100.21,100.12,100.22,101.07,100.79,100.62,100.53
5,май,101.34,101.76,100.97,101.47,101.38,100.79,100.60,100.49,101.01,...,100.46,100.35,100.35,100.39,100.46,100.44,99.68,101.13,101.53,101.34
6,июнь,103.02,102.53,101.64,101.22,100.89,100.92,100.68,100.61,101.10,...,100.56,100.68,100.74,100.60,100.12,100.76,100.88,100.83,101.06,100.59
7,июль,103.77,102.90,101.79,101.91,101.31,100.91,100.61,100.64,100.91,...,101.65,101.59,101.28,100.93,100.99,100.88,101.41,100.48,102.86,102.69
8,август,103.00,102.26,102.53,100.73,100.98,100.77,100.75,100.53,100.94,...,100.33,100.37,100.27,100.18,100.37,100.32,100.05,99.68,100.01,99.38
9,сентябрь,102.77,102.55,102.76,100.89,100.63,100.90,100.45,100.44,101.04,...,100.09,100.12,100.24,99.77,99.58,100.00,100.51,100.61,100.51,NaN
10,октябрь,102.40,101.91,102.51,100.80,101.20,100.69,100.41,100.13,100.01,...,99.74,99.80,99.94,99.82,99.87,100.01,100.24,100.48,100.21,NaN


In [156]:
df_melted = load_melted_ipc_data(IpcType.SERVICES)
df_melted.head()

,date,cpi
0,2000-01-01,103.37
1,2000-02-01,102.97
2,2000-03-01,101.45
3,2000-04-01,102.14
4,2000-05-01,101.34


In [157]:
plot_2d_interactive(
    x=df_melted['date'],
    y=df_melted['cpi'],
    xlabel="Дата",
    ylabel="Отношение к предыдущему месяцу (в процентах)",
    title=f"Индекс потребительских цен в России на услуги (1991–2025)"
)

In [158]:
df_melted = load_normalized_ipc_data(IpcType.SERVICES, base_year='2000-01-01')
df_melted

,date,cpi,cumulative_cpi,real_ruble
0,2000-01-01,103.37,1.0337,1.0
1,2000-02-01,102.97,1.064401,0.971157
2,2000-03-01,101.45,1.079835,0.957276
3,2000-04-01,102.14,1.102943,0.93722
4,2000-05-01,101.34,1.117723,0.924827
...,...,...,...,...
303,2025-04-01,100.53,21.846532,0.047316
304,2025-05-01,101.34,22.139276,0.046691
305,2025-06-01,100.59,22.269898,0.046417
306,2025-07-01,102.69,22.868958,0.045201


In [159]:
plot_2d_interactive(
    x=df_melted['date'],
    y=df_melted['real_ruble'],
    xlabel="Дата",
    ylabel="Реальная покупательная способность (в ценах 1 января 2000 г.)",
    title="Покупательная способность рубля относительно базового уровня на услуги (2000 = 1.0)"
)